In [1]:
import os
import pandas as pd
import statsmodels.api as sm
import glob


In [3]:
import pandas as pd
import statsmodels.api as sm
import os
import glob

# 读取 CH3_factors 数据
CH3_factors = pd.read_excel('C:\\Users\\86183\\Desktop\\资产管理大作业\\Alpha计算\\CH3_factors_monthly_202410.xlsx')

# 定义一个函数来执行合并与回归
def process_and_regress(anomaly_file, CH3_factors):
    # 读取 anomaly 文件
    anomaly_data = pd.read_csv(anomaly_file)

    # 2.1. 转换 anomaly 文件中的日期格式
    try:
        anomaly_data['year_month'] = pd.to_datetime(anomaly_data['year_month'], format='%Y-%m', errors='coerce').dt.strftime('%Y%m')
        # 处理转换失败的行
        if anomaly_data['year_month'].isnull().any():
            print(f"警告：文件 {anomaly_file} 中有无效日期，已跳过这些行")
            anomaly_data = anomaly_data.dropna(subset=['year_month'])  # 删除无效日期的行
    except Exception as e:
        print(f"日期转换错误 in file {anomaly_file}: {e}")
        return None

    # 2.2. 转换 CH3_factors 中的日期格式
    try:
        CH3_factors['mnthdt'] = pd.to_datetime(CH3_factors['mnthdt'], format='%Y%m%d', errors='coerce')
        CH3_factors['month'] = CH3_factors['mnthdt'].dt.strftime('%Y%m')  # 提取月份
        # 处理转换失败的行
        if CH3_factors['month'].isnull().any():
            print(f"警告：CH3_factors 中有无效日期，已跳过这些行")
            CH3_factors = CH3_factors.dropna(subset=['month'])  # 删除无效日期的行
    except Exception as e:
        print(f"日期转换错误 in CH3_factors: {e}")
        return None

    # 计算自变量 X：SMB, VMG, mktrf - rf_mon
    CH3_factors['mktrf_rf_mon'] = CH3_factors['mktrf'] - CH3_factors['rf_mon']
    CH3_factors = CH3_factors[['month', 'SMB', 'VMG', 'mktrf_rf_mon']]  # 保留月份作为匹配列

    # 2.3. 合并数据：按月份（year_month 和 month）进行合并
    merged_data = pd.merge(anomaly_data, CH3_factors, left_on='year_month', right_on='month', how='inner')

    # 2.4. 删除含有缺失值的行（仅删除缺失值的行，不跳过整个文件）
    merged_data = merged_data.dropna(subset=['return_spread_mv', 'SMB', 'VMG', 'mktrf_rf_mon'])

    # 检查合并后的数据
    if len(merged_data) == 0:
        print(f"合并后没有有效数据，文件：{anomaly_file}，跳过该文件。")
        return None

    # 定义 Y（因变量）和 X（自变量）
    Y = merged_data['return_spread_mv']
    X = merged_data[['SMB', 'VMG', 'mktrf_rf_mon']]

    # 添加常数项（截距）到 X 自变量中
    X = sm.add_constant(X)  # 添加常数项（截距）

    # 进行回归分析
    model = sm.OLS(Y, X)  # 创建 OLS 回归模型
    results = model.fit()  # 拟合模型

    # 获取回归模型的常数项和 p 值
    const_value = results.params['const']
    const_p_value = results.pvalues['const']

    # 其他因子的系数和 p 值
    factors = ['SMB', 'VMG', 'mktrf_rf_mon']
    factor_results = []

    # 从文件名中提取因子名，并按顺序列出因子和相关结果
    factor_name = os.path.basename(anomaly_file).replace('.csv', '')  # 从文件名提取因子名

    factor_results.append({
        '因子名': factor_name,
        '常数项的数值': const_value,
        '常数项的 p 值': const_p_value
    })

    # 打印回归结果
    print(f"回归结果：{anomaly_file}")
    print(results.summary())

    # 保存回归结果到文件
    output_folder = 'C:\\Users\\86183\\Desktop\\资产管理大作业\\Alpha计算\\Python-合并后的文档和源代码\\回归结果\\anomaly-CH3'
    
    # 确保目标文件夹存在，如果不存在则创建
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 保存回归结果为 CSV 文件（每个文件单独保存）
    output_file = os.path.join(output_folder, f"{os.path.basename(anomaly_file).replace('.csv', '')}_regression_result.csv")
    result_df = pd.DataFrame(factor_results)
    result_df.to_csv(output_file, index=False)
    print(f"回归结果已保存为：{output_file}")

    # 返回回归分析的结果
    return factor_results

# 批量处理文件
folder_path = 'C:\\Users\\86183\\Desktop\\资产管理大作业\\Alpha计算\\所有异象收益率数据（多空修正版）\\return'

# 获取所有 CSV 文件路径
anomaly_files = glob.glob(os.path.join(folder_path, "*.csv"))

# 创建一个列表来保存每个文件的回归结果
regression_results = []

# 遍历所有文件并处理
for anomaly_file in anomaly_files:
    result = process_and_regress(anomaly_file, CH3_factors)
    if result:
        regression_results.append(result)

# 保存所有回归结果到一个 CSV 文件
all_results_df = pd.DataFrame([item for sublist in regression_results for item in sublist])  # 扁平化列表
all_results_df.to_csv('C:\\Users\\86183\\Desktop\\资产管理大作业\\Alpha计算\\output\\anomaly-CH3.csv', index=False)

print("所有回归结果已保存。")


回归结果：C:\Users\86183\Desktop\资产管理大作业\Alpha计算\所有异象收益率数据（多空修正版）\return\ami12.csv
                            OLS Regression Results                            
Dep. Variable:       return_spread_mv   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     228.0
Date:                Sun, 15 Dec 2024   Prob (F-statistic):           8.48e-64
Time:                        22:53:33   Log-Likelihood:                 416.64
No. Observations:                 201   AIC:                            -825.3
Df Residuals:                     197   BIC:                            -812.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

回归结果：C:\Users\86183\Desktop\资产管理大作业\Alpha计算\所有异象收益率数据（多空修正版）\return\ami12.csv
                            OLS Regression Results                            
Dep. Variable:       return_spread_mv   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.485
Date:                Wed, 11 Dec 2024   Prob (F-statistic):              0.220
Time:                        23:41:17   Log-Likelihood:                 1180.5
No. Observations:                 201   AIC:                            -2353.
Df Residuals:                     197   BIC:                            -2340.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

In [4]:
import pandas as pd
import statsmodels.api as sm
import os
import glob

# 读取 CH4_factors 数据
CH4_factors = pd.read_excel('C:\\Users\\86183\\Desktop\\资产管理大作业\\Alpha计算\\CH4_factors_monthly_202410.xlsx')

# 定义一个函数来执行合并与回归
def process_and_regress(anomaly_file, CH4_factors):
    # 读取 anomaly 文件
    anomaly_data = pd.read_csv(anomaly_file)

    # 2.1. 转换 anomaly 文件中的日期格式
    try:
        anomaly_data['year_month'] = pd.to_datetime(anomaly_data['year_month'], format='%Y-%m', errors='coerce').dt.strftime('%Y%m')
        # 处理转换失败的行
        if anomaly_data['year_month'].isnull().any():
            print(f"警告：文件 {anomaly_file} 中有无效日期，已跳过这些行")
            anomaly_data = anomaly_data.dropna(subset=['year_month'])  # 删除无效日期的行
    except Exception as e:
        print(f"日期转换错误 in file {anomaly_file}: {e}")
        return None

    # 2.2. 转换 CH4_factors 中的日期格式
    try:
        CH4_factors['mnthdt'] = pd.to_datetime(CH4_factors['mnthdt'], format='%Y%m%d', errors='coerce')
        CH4_factors['month'] = CH4_factors['mnthdt'].dt.strftime('%Y%m')  # 提取月份
        # 处理转换失败的行
        if CH4_factors['month'].isnull().any():
            print(f"警告：CH4_factors 中有无效日期，已跳过这些行")
            CH4_factors = CH4_factors.dropna(subset=['month'])  # 删除无效日期的行
    except Exception as e:
        print(f"日期转换错误 in CH4_factors: {e}")
        return None

    # 计算自变量 X：SMB, VMG, mktrf - rf_mon
    CH4_factors['mktrf_rf_mon'] = CH4_factors['mktrf'] - CH4_factors['rf_mon']
    CH4_factors = CH4_factors[['month', 'SMB', 'VMG', 'mktrf_rf_mon']]  # 保留月份作为匹配列

    # 2.3. 合并数据：按月份（year_month 和 month）进行合并
    merged_data = pd.merge(anomaly_data, CH4_factors, left_on='year_month', right_on='month', how='inner')

    # 2.4. 删除含有缺失值的行（仅删除缺失值的行，不跳过整个文件）
    merged_data = merged_data.dropna(subset=['return_spread_mv', 'SMB', 'VMG', 'mktrf_rf_mon'])

    # 检查合并后的数据
    if len(merged_data) == 0:
        print(f"合并后没有有效数据，文件：{anomaly_file}，跳过该文件。")
        return None

    # 定义 Y（因变量）和 X（自变量）
    Y = merged_data['return_spread_mv']
    X = merged_data[['SMB', 'VMG', 'mktrf_rf_mon']]

    # 添加常数项（截距）到 X 自变量中
    X = sm.add_constant(X)  # 添加常数项（截距）

    # 进行回归分析
    model = sm.OLS(Y, X)  # 创建 OLS 回归模型
    results = model.fit()  # 拟合模型

    # 获取回归模型的常数项和 p 值
    const_value = results.params['const']
    const_p_value = results.pvalues['const']

    # 其他因子的系数和 p 值
    factors = ['SMB', 'VMG', 'mktrf_rf_mon']
    factor_results = []

    # 从文件名中提取因子名，并按顺序列出因子和相关结果
    factor_name = os.path.basename(anomaly_file).replace('.csv', '')  # 从文件名提取因子名

    factor_results.append({
        '因子名': factor_name,
        '常数项的数值': const_value,
        '常数项的 p 值': const_p_value
    })

    # 打印回归结果
    print(f"回归结果：{anomaly_file}")
    print(results.summary())

    # 保存回归结果到文件
    output_folder = 'C:\\Users\\86183\\Desktop\\资产管理大作业\\Alpha计算\\Python-合并后的文档和源代码\\回归结果\\anomaly-CH4'
    
    # 确保目标文件夹存在，如果不存在则创建
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 保存回归结果为 CSV 文件（每个文件单独保存）
    output_file = os.path.join(output_folder, f"{os.path.basename(anomaly_file).replace('.csv', '')}_regression_result.csv")
    result_df = pd.DataFrame(factor_results)
    result_df.to_csv(output_file, index=False)
    print(f"回归结果已保存为：{output_file}")

    # 返回回归分析的结果
    return factor_results

# 批量处理文件
folder_path = 'C:\\Users\\86183\\Desktop\\资产管理大作业\\Alpha计算\\所有异象收益率数据（多空修正版）\\return'

# 获取所有 CSV 文件路径
anomaly_files = glob.glob(os.path.join(folder_path, "*.csv"))

# 创建一个列表来保存每个文件的回归结果
regression_results = []

# 遍历所有文件并处理
for anomaly_file in anomaly_files:
    result = process_and_regress(anomaly_file, CH4_factors)
    if result:
        regression_results.append(result)

# 保存所有回归结果到一个 CSV 文件
all_results_df = pd.DataFrame([item for sublist in regression_results for item in sublist])  # 扁平化列表
all_results_df.to_csv('C:\\Users\\86183\\Desktop\\资产管理大作业\\Alpha计算\\output\\anomaly-CH4.csv', index=False)

print("所有回归结果已保存。")


回归结果：C:\Users\86183\Desktop\资产管理大作业\Alpha计算\所有异象收益率数据（多空修正版）\return\ami12.csv
                            OLS Regression Results                            
Dep. Variable:       return_spread_mv   R-squared:                       0.772
Model:                            OLS   Adj. R-squared:                  0.768
Method:                 Least Squares   F-statistic:                     222.3
Date:                Sun, 15 Dec 2024   Prob (F-statistic):           5.74e-63
Time:                        22:55:48   Log-Likelihood:                 414.68
No. Observations:                 201   AIC:                            -821.4
Df Residuals:                     197   BIC:                            -808.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------